In [31]:
## Import packages and initial data ##

from sklearn.model_selection import train_test_split
import pandas
import numpy
import warnings
from sklearn import preprocessing
from sklearn import metrics
from sklearn.linear_model import LinearRegression

warnings.filterwarnings('ignore')

pandas.set_option('display.max_columns', 500)
pandas.set_option('display.max_rows', 500)
data = pandas.read_csv('ncaab_box_scores.csv')
data = data[data.columns[-47:]]
data['ranked'] = numpy.where(data['team_rank']>0, 1, 0)
data['ranked_opp'] = numpy.where(data['opp_rank']>0, 1, 0)
data['game_date'] = pandas.to_datetime(data['game_date'])
data = pandas.get_dummies(data,prefix=['team_home_away'], columns=['team_home_away'])

## Transform and order columns ##

drops = ['unique_game','opponent','team_def_rating','opp_def_rating','team_off_rating',
        'opp_off_rating','team_rank','opp_rank','team_eff_fg_pct', 'team_fg_pct','opp_eff_fg_pct', 'opp_fg_pct']
data_transform = data.drop(drops, 1).dropna()
datacols = ['team','team1_assist_pct', 'team1_block_pct',
       'team1_def_reb_pct', 'team1_ft_rate',
       'team1_ft_pct', 'team1_off_reb_pct', 'team1_pts',
       'team1_stl_pct', 'team1_3pt_rate', 'team1_3pt_pct',
       'team1_reb_pct', 'team1_true_shoot_pct', 'team1_to_pct', 'team1_2pt_rate',
       'team1_2pt_pct','team2_assist_pct', 'team2_block_pct',
       'team2_def_reb_pct', 'team2_ft_rate',
       'team2_ft_pct', 'team2_off_reb_pct', 'team2_pts',
       'team2_stl_pct', 'team2_3pt_rate', 'team2_3pt_pct',
       'team2_reb_pct', 'team2_true_shoot_pct', 'team2_to_pct', 'team2_2pt_rate',
       'team2_2pt_pct','team1_pace', 'team1_win_pct','game_date', 'team1_ranked_pct', 'team1_ranked_opp_pct',
       'team1_Away', 'team1_Home']
data_transform.columns = datacols

## Create dictionary of matchups to predict on ##
## Order of dictionary: home team name, 1 for home, 1 for ranked opp, away team name, 0 for away, 1 for ranked opp ##
###################Opportunity to add date filter here now that I am passing through date!!!
matchup_dict = [('PENN-STATE',1,0,'ILLINOIS',0,1),
                ('BAYLOR',1,1,'KANSAS',0,1),
               ('NORTH-CAROLINA-STATE',1,1,'DUKE',0,0)]
matchup_concat = pandas.DataFrame()

## Loop through list to get matchups in correct form ##

for teamname1,home1,rankopp1,teamname2,home2,rankopp2 in matchup_dict:
    team1 = pandas.DataFrame(data_transform[numpy.logical_and(numpy.logical_and(data_transform['team']==teamname1,data_transform['team1_Home']==home1),data_transform['team1_ranked_opp_pct']==rankopp1)].mean())
    team2 = pandas.DataFrame(data_transform[numpy.logical_and(numpy.logical_and(data_transform['team']==teamname2,data_transform['team1_Home']==home2),data_transform['team1_ranked_opp_pct']==rankopp2)].mean())
    team1 = team1.transpose()
    team2 = team2.transpose()
    team1['team1'] = teamname1
    team2['team2'] = teamname2
    matchup = pandas.concat([team1,team2],axis=1)
    matchup_concat = pandas.concat([matchup_concat,matchup],axis=0)

matchup_concat = matchup_concat.iloc[:,numpy.r_[0:15,30:52,73:74]]

matchup_concat_cols = ['team1_assist_pct', 'team1_block_pct',
       'team1_def_reb_pct', 'team1_ft_rate',
       'team1_ft_pct', 'team1_off_reb_pct', 'team1_pts',
       'team1_stl_pct', 'team1_3pt_rate', 'team1_3pt_pct',
       'team1_reb_pct', 'team1_true_shoot_pct', 'team1_to_pct', 'team1_2pt_rate',
       'team1_2pt_pct', 'team1_pace', 'team1_win_pct', 'team1_ranked_pct', 'team1_ranked_opp_pct',
       'team1_Away', 'team1_Home','team1','team2_assist_pct', 'team2_block_pct',
       'team2_def_reb_pct', 'team2_ft_rate',
       'team2_ft_pct', 'team2_off_reb_pct', 'team2_pts',
       'team2_stl_pct', 'team2_3pt_rate', 'team2_3pt_pct',
       'team2_reb_pct', 'team2_true_shoot_pct', 'team2_to_pct', 'team2_2pt_rate',
       'team2_2pt_pct', 'team2']
matchup_concat.columns = matchup_concat_cols

regcols = ['team1_assist_pct', 'team1_block_pct',
       'team1_def_reb_pct', 'team1_ft_rate',
       'team1_ft_pct', 'team1_off_reb_pct', 'team1_pts',
       'team1_stl_pct', 'team1_3pt_rate', 'team1_3pt_pct',
       'team1_reb_pct', 'team1_true_shoot_pct', 'team1_to_pct', 'team1_2pt_rate',
       'team1_2pt_pct','team1_pace','team1_win_pct', 'team1_ranked_pct', 'team1_ranked_opp_pct',
       'team1_Away', 'team1_Home','team2_assist_pct', 'team2_block_pct',
       'team2_def_reb_pct', 'team2_ft_rate',
       'team2_ft_pct', 'team2_off_reb_pct', 'team2_pts',
       'team2_stl_pct', 'team2_3pt_rate', 'team2_3pt_pct',
       'team2_reb_pct', 'team2_true_shoot_pct', 'team2_to_pct', 'team2_2pt_rate',
       'team2_2pt_pct']

regdata = data_transform[regcols]

## Prepare LinReg variables ##

x = regdata.drop(['team1_pts','team2_pts','team1_win_pct','team1_ranked_pct',
       'team1_ranked_opp_pct', 'team1_Away', 'team1_Home'],1)
y = regdata['team1_pts']
y_2 = regdata['team2_pts']
new_x = matchup_concat.drop(['team1_pts','team2_pts','team1_win_pct', 'team1_ranked_pct',
       'team1_ranked_opp_pct', 'team1_Away', 'team1_Home', 'team1','team2'],1)

linreg_team1 = LinearRegression()
linreg_team2 = LinearRegression()

## Fit the models ##

model_team1 = linreg_team1.fit(x, y)
model_team2 = linreg_team2.fit(x, y_2)

## Get point predictions for the matchup ##

y_predict_matchup = model_team1.predict(new_x)
matchup_concat['team1_pts_est'] = y_predict_matchup
y_predict_matchup_2 = model_team2.predict(new_x)
matchup_concat['team2_pts_est'] = y_predict_matchup_2

matchup_concat

,team1_assist_pct,team1_block_pct,team1_def_reb_pct,team1_ft_rate,team1_ft_pct,team1_off_reb_pct,team1_pts,team1_stl_pct,team1_3pt_rate,team1_3pt_pct,team1_reb_pct,team1_true_shoot_pct,team1_to_pct,team1_2pt_rate,team1_2pt_pct,team1_pace,team1_win_pct,team1_ranked_pct,team1_ranked_opp_pct,team1_Away,team1_Home,team1,team2_assist_pct,team2_block_pct,team2_def_reb_pct,team2_ft_rate,team2_ft_pct,team2_off_reb_pct,team2_pts,team2_stl_pct,team2_3pt_rate,team2_3pt_pct,team2_reb_pct,team2_true_shoot_pct,team2_to_pct,team2_2pt_rate,team2_2pt_pct,team2,team1_pts_est,team2_pts_est
0,0.544923,0.147308,0.783615,0.319385,0.690769,0.293077,77.076923,0.121231,0.395154,0.296538,0.540615,0.534692,0.113846,0.604846,0.549077,73.215385,0.846154,0.461538,0.0,0.0,1.0,PENN-STATE,0.497750,0.038500,0.693250,0.263250,0.701500,0.271250,62.000000,0.061750,0.326000,0.249500,0.473250,0.465750,0.140000,0.674000,0.46025,ILLINOIS,83.350379,78.127792
0,0.569250,0.160750,0.707250,0.296000,0.649750,0.331750,68.250000,0.107250,0.380500,0.346000,0.517250,0.519250,0.127500,0.619500,0.476000,68.700000,1.000000,1.000000,1.0,0.0,1.0,BAYLOR,0.516000,0.148333,0.702667,0.389667,0.560667,0.247667,59.666667,0.143333,0.242333,0.320333,0.508667,0.505667,0.166667,0.757667,0.49300,KANSAS,78.515722,59.195538
0,0.300000,0.114000,0.708000,0.279000,0.706000,0.278000,57.000000,0.100000,0.377000,0.217000,0.450000,0.413000,0.110000,0.623000,0.395000,70.200000,0.000000,0.000000,1.0,0.0,1.0,NORTH-CAROLINA-STATE,0.508625,0.095750,0.729500,0.371000,0.731625,0.327625,82.750000,0.106500,0.301875,0.326250,0.530750,0.568750,0.121250,0.698125,0.55400,DUKE,56.351571,84.446635
